In [24]:
import pandas as pd
import numpy as np
import pdfplumber

#!pip install cv2

In [2]:
pdf1_path="E:\\new_Desktop\\8月求職者複試題目 (1)\\8月複試內容\\1 爬蟲 PDF\\1 萬能科大.pdf"
pdf2_path="E:\\new_Desktop\\8月求職者複試題目 (1)\\8月複試內容\\1 爬蟲 PDF\\2 淡江大學.pdf"
pdf3_path="E:\\new_Desktop\\8月求職者複試題目 (1)\\8月複試內容\\1 爬蟲 PDF\\3 義守大學.pdf"

# 遭遇問題

遇到一個問題，在我的爬蟲條件中，一般生正取、一般生正取這些關鍵字是很重要的

但我直接打字的一般生正取，跟我從pdf的text中複製過來的一般生正取，似乎編碼不同，因此我還要重新去將其貼上才能處理

但是處理成table的方式大致相同。

In [6]:
class pdf_crawler:
    
    def __init__(self,pdf_file_path):
        text=""
        pdf=pdfplumber.open(pdf_file_path)
        for i in range(len(pdf.pages)):
            text=text+pdf.pages[i].extract_text()
            
        self.text=text
    @staticmethod
    def delete_white(li):
        """
        li為list 來去除當中空白的東西。
        """
        df=[]
        for h in range(len(li)):
            if "Ｏ" in li[h]:
                df.append(li[h])
        return df
    
    
    def get_df_list(self):
        pdf_text=self.text.replace('\n', '  ').replace('--', '系所--')
        pdf_text=pdf_text.split('系所--')
        pdf_text = pdf_text[1:]
        total = []
        names = []
        for i in pdf_text:
            if "⼀般⽣正取" in i:
                postive = True
            else:
                postive = False
            if "⼀般⽣備取" in i:
                alternate = True
            else:
                alternate = False
            if "原住⺠正取" in i:
                ab_postive = True
            else:
                ab_postive = False
            if "原住⺠備取" in i:
                ab_alternate = True
            else:
                ab_alternate = False
            if "離島生正取" in i:
                out_postive = True
            else:
                out_postive = False
            if "離島生備取" in i:
                out_alternate = True
            else:
                out_alternate = False
            name=i.split("  ")[0]
            all_df=[]
             #處理正常正備取
            if postive:
                postive_name = i.split("⼀般⽣正取")[1].split("⼀般⽣備取")[0].split(" ")[4:]
                if len(postive_name) > 1:
                    postive_name = i.split("⼀般⽣正取")[1].split("⼀般⽣備取")[0].split(" ")[4:][:-1]
                else:
                    postive_name = i.split("⼀般⽣正取")[1].split("⼀般⽣備取")[0].split(" ")[4:]
                #print(postive_name)
                postive_df=self.delete_white(li=postive_name)
                #不想要有空白格 ，用這個把它去掉
#                 for h in range(len(postive_name)):
#                     if "Ｏ" in postive_name[h]:
#                         postive_df.append(postive_name[h])
                postive_df = pd.DataFrame(postive_df, columns=[[f'{name}_一般生正取']]).reset_index()  #取得正取的dataframe
                # print(postive_df)
                # print(type(postive_df))
                all_df.append(postive_df)
            if alternate:
                #print(i.split("一般生備取")[1])
                alternate_name = i.split("⼀般⽣備取")[1].split('[')
                alternate_name = alternate_name[1:]
                alternate_name[-1] = alternate_name[-1].split("\u3000")[0]
                alternate_df = []
                #分字串的時候發現，朝陽大學的，同時有備取跟原住民備取時會出錯，家條件調整
                for z in alternate_name:
                    if "原住⺠正取" in z:
                        c_index = alternate_name.index(z)
                        # print(c_index)
                        alternate_name = alternate_name[:c_index + 1]
                        alternate_name[-1] = alternate_name[-1].split(" ")[0]
                for m in alternate_name:
                    elm = m.split("]")[1]
                    alternate_df.append(elm)
                alternate_df = pd.DataFrame(alternate_df, columns=[[f'{name}_⼀般⽣備取']]).reset_index()
                # print(alternate_df)
                # print(type(postive_df))
                all_df.append(alternate_df)
                #處理原住民生
            if ab_postive:
                ab_postive_name = i.split("原住⺠正取")[1].split('原住⺠備取')[0].split(" ")[4:]
                if len(ab_postive_name) > 1:
                    ab_postive_name = i.split("原住⺠正取")[1].split('原住⺠備取')[0].split(" ")[4:][:-1]
                else:
                    ab_postive_name = i.split("原住⺠正取")[1].split('原住⺠備取')[0].split(" ")[4:]
                ab_postive_df = self.delete_white(li=ab_postive_name)
#                 for p in ab_postive_name:
#                     if len(p) <= 12:
#                         ab_postive_df.append(p)
#                     else:
#                         p1 = p[0:11]
#                         p2 = p[11:]
#                         ab_postive_df.append(p1)
#                         ab_postive_df.append(p2)

                ab_postive_df = pd.DataFrame(ab_postive_df, columns=[[f'{name}_原住⺠正取']]).reset_index()
                all_df.append(ab_postive_df)
            if ab_alternate:
                ab_alternate_name = i.split("原住⺠備取")[1].split('[')
                ab_alternate_name = ab_alternate_name[1:]
                ab_alternate_name[-1] = ab_alternate_name[-1].split("\u3000")[0]
                ab_alternate_df = []
                for l in ab_alternate_name:
                    elm = l.split("]")[1]
                    ab_alternate_df.append(elm)
                ab_alternate_df = pd.DataFrame(ab_alternate_df, columns=[[f'{name}_原住⺠備取']]).reset_index()
                all_df.append(ab_alternate_df)
            wholedf = pd.DataFrame()
            for o in range(len(all_df)):
                if len(all_df) == 0:
                    wholedf = pd.DataFrame(columns=[[f'{name}']])
                elif o == 0 and len(all_df) != 1:
                    wholedf = pd.merge(all_df[o], all_df[o + 1], how='outer')
                elif o == 0 and len(all_df) == 1:
                    wholedf = all_df[o]
                else:
                    wholedf = pd.merge(wholedf, all_df[o], how='outer')

            total.append(wholedf)
            names.append(name)
        total_return = [total, names]
        return total_return
#         return all_df
        
        

In [7]:
pdf1=pdf_crawler(pdf_file_path=pdf1_path).get_df_list()

In [8]:
for u in range(len(pdf1[0])):
    name=pdf1[1][u]
    print(f'{u}_{name}')
    pdf1[0][u].to_csv(f'萬能科技大學_{name}.csv')

0_室內設計與營建科技系室內設計與管理組(機械群)
1_室內設計與營建科技系室內設計與管理組(⼟⽊與建築群)
2_室內設計與營建科技系室內設計與管理組(設計群)
3_資訊⼯程系電機控制組(電機與電⼦群電機類)
4_資訊⼯程系電機控制組(電機與電⼦群資電類)
5_資訊⼯程系遊戲設計與競技組(電機與電⼦群資電類)
6_資訊⼯程系遊戲設計與競技組(設計群)
7_資訊⼯程系遊戲設計與競技組(商業與管理群)
8_航空光機電系(機械群)
9_航空光機電系(動⼒機械群)
10_航空光機電系(電機與電⼦群電機類)
11_精密機械與⼯業⼯程系(機械群)
12_精密機械與⼯業⼯程系(動⼒機械群)
13_⾞輛⼯程系(動⼒機械群)
14_化妝品應⽤與管理系(家政群幼保類)
15_化妝品應⽤與管理系(家政群⽣活應⽤類)
16_化妝品應⽤與管理系(餐旅群)
17_時尚造型設計系時尚造型設計組(家政群⽣活應⽤類)
18_時尚造型設計系時尚造型設計組(餐旅群)
19_時尚造型設計系時尚造型設計組(藝術群影視類)
20_時尚造型設計系時尚表演藝術組(家政群⽣活應⽤類)
21_時尚造型設計系時尚表演藝術組(餐旅群)
22_時尚造型設計系時尚表演藝術組(藝術群影視類)
23_商業設計系(設計群)
24_商業設計系(商業與管理群)
25_商業設計系(家政群⽣活應⽤類)
26_觀光與休閒事業管理系(商業與管理群)
27_觀光與休閒事業管理系(外語群英語類)
28_觀光與休閒事業管理系(餐旅群)
29_餐飲管理系(餐旅群)
30_旅館管理系(商業與管理群)
31_旅館管理系(外語群英語類)
32_旅館管理系(餐旅群)
33_航空暨運輸服務管理系(商業與管理群)
34_航空暨運輸服務管理系(外語群英語類)
35_航空暨運輸服務管理系(餐旅群)
36_航空暨運輸服務管理系國際服務組(家政群⽣活應⽤類)
37_航空暨運輸服務管理系國際服務組(外語群英語類)
38_航空暨運輸服務管理系國際服務組(外語群⽇語類)
39_⾏銷與流通管理系(動⼒機械群)
40_⾏銷與流通管理系(商業與管理群)
41_⾏銷與流通管理系(餐旅群)
42_⾏銷與流通管理系航空暨運籌管理組(商業與管理群)
43_⾏銷與流通管理系航空暨運籌管理組(外語群英語類)
44_企業管理系(動⼒機械群)
45_企業管理系(商業與管理群)
46_企業管理系(餐

In [9]:
pdf2=pdf_crawler(pdf_file_path=pdf2_path).get_df_list()

In [10]:
for u in range(len(pdf2[0])):
    name=pdf2[1][u]
    print(f'{u}_{name}')
    pdf2[0][u].to_csv(f'淡江大學_{name}.csv')

0_資訊傳播學系(設計群)
1_化學⼯程與材料⼯程學系(化⼯群)
2_會計學系(商業與管理群)
3_資訊管理學系(資管類)
4_英文學系(外語群英語類)
5_⻄班牙語文學系(外語群⽇語類)
6_俄國語文學系(外語群⽇語類)
7_電機⼯程學系電機通訊組(電機與電⼦群電機類)
8_化學⼯程與材料⼯程學系(食品群)


In [21]:
class origin_encode_cralwer:
    def __init__(self, pdf_file_path):
        text=""
        pdf=pdfplumber.open(pdf_file_path)
        for i in range(len(pdf.pages)):
            text=text+pdf.pages[i].extract_text()
            
        self.text=text

    def get_df_list(self):
        pdf_text=self.text.replace('\n', '  ').replace('--', '系所--')
        pdf_text=pdf_text.split('系所--')
        pdf_text = pdf_text[1:]
        total = []
        names = []
        for i in pdf_text:
            if "一般生正取" in i:
                postive = True
            else:
                postive = False
            if "一般生備取" in i:
                alternate = True
            else:
                alternate = False
            if "原住民正取" in i:
                ab_postive = True
            else:
                ab_postive = False
            if "原住民備取" in i:
                ab_alternate = True
            else:
                ab_alternate = False
            if "離島生正取" in i:
                out_postive = True
            else:
                out_postive = False
            if "離島生備取" in i:
                out_alternate = True
            else:
                out_alternate = False
            # print(f'是否有正取:{postive}')
            # print(f'是否有備取:{alternate}')
            # print(i.split("  ")[0])  #取得系所名稱
            name = i.split("  ")[0]
            all_df = []
#             print(name)
            #處理正常正備取
            if postive:
                #print(f'正取_{i.split("一般生正取")[1].split("一般生備取")[0]}')

                postive_name = i.split("一般生正取")[1].split("一般生備取")[0].split("  ")[4:]
                if len(postive_name) > 1:
                    postive_name = i.split("一般生正取")[1].split("一般生備取")[0].split("  ")[4:][:-1]
                else:
                    postive_name = i.split("一般生正取")[1].split("一般生備取")[0].split("  ")[4:]
#                 print(postive_name[:-10])
#                 postive_name=postive_name[:-10]
                postive_df = pd.DataFrame(postive_name, columns=[[f'{name}_一般生正取']]).reset_index()  #取得正取的dataframe
#                 print(postive_df)
                # print(type(postive_df))
                all_df.append(postive_df)
            if alternate:
                #print(i.split("一般生備取")[1])
                alternate_name = i.split("一般生備取")[1].split('[')
                alternate_name = alternate_name[1:]
                alternate_name[-1] = alternate_name[-1].split("  ")[0]
                for strip in range(len(alternate_name)):
                    alternate_name[strip]=alternate_name[strip].rstrip("")
#                 print(alternate_name)
                #分字串的時候發現，朝陽大學的，同時有備取跟原住民備取時會出錯，家條件調整
#                 for z in alternate_name:
#                     if "原住民正取" in z:
#                         c_index = alternate_name.index(z)
#                         # print(c_index)
#                         alternate_name = alternate_name[:c_index + 1]
#                         alternate_name[-1] = alternate_name[-1].split(" ")[0]
                for m in range(len(alternate_name)):
                    alternate_name[m] = alternate_name[m].replace("]"," ")
#                     alternate_df.append(elm)
                alternate_df = pd.DataFrame(alternate_name, columns=[[f'{name}_一般生備取']]).reset_index()
#                 print(alternate_df)
                # print(type(postive_df))
                all_df.append(alternate_df)

            #處理原住民生
            if ab_postive:
                ab_postive_name = i.split("原住民正取")[1].split('原住民備取')[0].split(" ")[4:]
                if len(ab_postive_name) > 1:
                    ab_postive_name = i.split("原住民正取")[1].split('原住民備取')[0].split(" ")[4:][:-1]
                else:
                    ab_postive_name = i.split("原住民正取")[1].split('原住民備取')[0].split(" ")[4:]
                ab_postive_df = []
                for p in ab_postive_name:
                    if len(p) <= 12:
                        ab_postive_df.append(p)
                    else:
                        p1 = p[0:11]
                        p2 = p[11:]
                        ab_postive_df.append(p1)
                        ab_postive_df.append(p2)
                ab_postive_df = pd.DataFrame(ab_postive_df, columns=[[f'{name}_原住民正取']]).reset_index()
                all_df.append(ab_postive_df)
            if ab_alternate:
                ab_alternate_name = i.split("原住民備取")[1].split('[')
                ab_alternate_name = ab_alternate_name[1:]
                ab_alternate_name[-1] = ab_alternate_name[-1].split("\u3000")[0]
                ab_alternate_df = []
                for l in ab_alternate_name:
                    elm = l.split("]")[1]
                    ab_alternate_df.append(elm)
                ab_alternate_df = pd.DataFrame(ab_alternate_df, columns=[[f'{name}_原住民備取']]).reset_index()
                all_df.append(ab_alternate_df)

            #處理離島生
            if out_postive:
                out_postive_name = i.split("離島生正取")[1].split('離島生備取')[0].split(" ")[4:]
                if len(out_postive_name) > 1:
                    out_postive_name = i.split("離島生正取")[1].split('離島生備取')[0].split(" ")[4:][:-1]
                else:
                    out_postive_name = i.split("離島生正取")[1].split('離島生備取')[0].split(" ")[4:]
                out_postive_df = []
                for p in out_postive_name:
                    if len(p) <= 12:
                        out_postive_df.append(p)
                    else:
                        p1 = p[0:11]
                        p2 = p[11:]
                        out_postive_df.append(p1)
                        out_postive_df.append(p2)
                out_postive_df = pd.DataFrame(out_postive_df, columns=[[f'{name}_離島生正取']]).reset_index()
                all_df.append(out_postive_df)
            if out_alternate:
                out_alternate_name = i.split("離島生備取")[1].split('[')[1:][-1].split("  ")[:-1]
                out_alternate_df = []
                print(out_alternate_name)
                for l in out_alternate_name:
                    elm = l.split("]")[1]
                    out_alternate_df.append(elm)

                out_alternate_df = pd.DataFrame(out_alternate_df, columns=[[f'{name}_離島生備取']]).reset_index()
                all_df.append(out_alternate_df)
            wholedf = pd.DataFrame()
            for o in range(len(all_df)):
                if len(all_df) == 0:
                    wholedf = pd.DataFrame(columns=[[f'{name}']])
                elif o == 0 and len(all_df) != 1:
                    wholedf = pd.merge(all_df[o], all_df[o + 1], how='outer')
                elif o == 0 and len(all_df) == 1:
                    wholedf = all_df[o]
                else:
                    wholedf = pd.merge(wholedf, all_df[o], how='outer')

            total.append(wholedf)
            names.append(name)
        total_return = [total, names]
        return total_return

In [22]:
pdf3=origin_encode_cralwer(pdf_file_path=pdf3_path).get_df_list()

In [23]:
for t in range(len(pdf3[0])):
    name=pdf3[1][t]
    print(f'{t}_{name}')
    pdf3[0][t].to_csv(f'義守大學_{name}.csv')
    

0_機械與自動化工程學系(機械群)
1_化學工程學系(化工群)
2_土木工程學系(土木與建築群)
3_創意商品設計學系(設計群)
4_企業管理學系(商業與管理群)
5_財務金融管理學系(商業與管理群)
6_國際商務學系(商業與管理群)
7_醫務管理學系(商業與管理群)
8_休閒事業管理學系(商業與管理群)
9_應用英語學系(外語群英語類)
10_應用日語學系(外語群日語類)
11_觀光學系(餐旅群)
12_餐旅管理學系(餐旅群)
13_廚藝學系(餐旅群)
14_大眾傳播學系(藝術群影視類)
15_國際觀光餐旅學系(餐旅群)
16_國際觀光餐旅學系(外語群英語類)
